In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import seaborn as sns

In [11]:
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LogisticRegressionCV

from sklearn.model_selection import cross_validate

from sklearn.pipeline import make_pipeline

In [2]:
# Custom modules

import sys
import os
ROOT_DIR = os.path.abspath('../')
sys.path.insert(0, ROOT_DIR + "/src")

# Approach

- Based on EDA results from notebook `01_eda`, the baseline we're going to use is 55% accuracy by predicting all customers being female.
- Given this dataset and a timeline of a few days, a reasonable goal is to build a classifier with 70% accuracy.

# Constants

In [15]:
TRAIN_INPUT_PATH = '../data/process/train_agg.csv'
TEST_INPUT_PATH = '../data/process/test_agg.csv'
LABEL_COL = 'gender_first'

# Load input

In [8]:
train_df = pd.read_csv(TRAIN_INPUT_PATH)

In [13]:
test_df = pd.read_csv(TEST_INPUT_PATH)

In [10]:
train_df

,step_count,step_mean,step_median,age_first,merchant_nunique,category_nunique,amount_mean,amount_median,amount_max,amount_min,amount_std,fraud_sum,fraud_mean,gender_first
0,131,107.786260,106.0,5,14,10,35.091908,26.14,323.64,0.80,36.863536,1,0.007634,M
1,109,124.532110,134.0,4,22,11,90.336239,35.13,3902.93,1.37,381.812802,5,0.045872,M
2,94,51.436170,49.5,4,10,7,38.154894,28.32,326.34,0.99,41.544414,0,0.000000,M
3,30,91.833333,94.5,1,11,6,204.754667,103.68,1260.94,1.05,306.240251,11,0.366667,M
4,131,107.702290,109.0,3,17,12,34.253282,29.94,197.30,0.59,27.820062,0,0.000000,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3275,175,89.891429,91.0,3,14,9,32.555771,26.63,241.58,0.17,30.641718,0,0.000000,M
3276,171,89.005848,89.0,2,11,10,61.211637,26.22,5527.37,0.41,421.233339,2,0.011696,M
3277,177,90.107345,91.0,2,8,6,32.273107,26.20,219.99,0.54,30.122381,0,0.000000,F
3278,142,104.992958,107.5,1,16,8,36.018944,26.00,476.43,0.12,48.619220,2,0.014085,F


# Generate X, y

In [16]:
X_train, y_train = train_df.drop(columns=[LABEL_COL]), train_df[LABEL_COL]
X_test, y_test = test_df.drop(columns=[LABEL_COL]), test_df[LABEL_COL]

# Transform

In [18]:
pt = PowerTransformer()

In [19]:
pt.fit(X_train)

PowerTransformer()

# Fit

In [36]:
clf = LogisticRegressionCV(cv=5, max_iter=1000)

In [37]:
pipeline = make_pipeline(pt, clf)

In [38]:
pipeline.fit(pt.transform(X_train), y_train)

Pipeline(steps=[('powertransformer', PowerTransformer()),
                ('logisticregressioncv',
                 LogisticRegressionCV(cv=5, max_iter=1000))])

# Evaluate

In [53]:
SCORING = ['roc_auc', 'f1_micro', 'f1_weighted', 'precision', 'recall']
train_score = cross_validate(pipeline, X_train, y_train, scoring=SCORING,
                             cv=5, return_train_score=True)

/home/quydv1/anaconda3/envs/banksim/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1254: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if pos_label not in present_labels:


ValueError: pos_label=1 is not a valid label: array(['F', 'M'], dtype='<U1')

# Predict

In [39]:
y_train_pred = pipeline.predict(X_train)

In [41]:
np.unique(y_train_pred, return_counts=True)

(array(['F'], dtype=object), array([3280]))

The fact that model always return 'F' indicates that model can not learn any patterns from data input `fe1`

In [40]:
y_test_pred = pipeline.predict(X_test)

# Archive

In [52]:
from sklearn.metrics import SCORERS
sorted(SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_weighted',
 'v_measure_score']